Neural Networks with TensorFlow (Follow Along)
===

We are going to learn how to build a TensorFlow implementation of an Artificial Neural Network. More especifically, we will learn how to classify a set of images using a Convolutional Neural Network. We will cover the following sections:  

1. A very simple MNIST Classifier
2. Visualizing our Results
3. tf.layers

**Note:** This document is part of the "Neural Networks with TensorFlow" workshop, which is meant to accompany these [slides](https://docs.google.com/presentation/d/185r0vPc4gzE_-4dOsjy0JmO8qk2M7BBm2Uhqg5tAF7E/edit?usp=sharing). The main source for this notebook is https://www.tensorflow.org/get_started/mnist/beginners. For more information please visit [AI for Everyone!](http://andresvourakis.com/ai-for-everyone)

A very simple MNIST Classifier
---

In this section we will create a **softmax regression** function that is a model for recognizing MNIST digits, based on looking at every pixel in the image.

![](https://www.tensorflow.org/images/MNIST.png)

In [2]:
# Imports

# Create Model

# Define Loss and Optimizer

# Train

# Test Trained Model

Visualizing our results
---

Let's dive a little deeper (tf.layers)
---


What if we wanted to build an Artificial Neural Network from scratch? Well, TensorFlow provides us with **tf.layers**, a set of high-level neural networks layers.

One of the most popular techniques used in improving the accuracy of image classification is Convolutional Neural Networks, let's see how one would go about using **tf.layers** to do that.